In [1]:
import sys
sys.path.append('/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage')

from main import parser_arguments, regional_extremes_method, local_extremes_method, regional_extremes_minicube

In [2]:
import numpy as np
from dask_jobqueue import SLURMCluster
from dask.distributed import Client as daskClient

In [3]:
# Set distributed scheduler as default
import dask
dask.config.set(scheduler='distributed')

cluster = SLURMCluster(
    queue='work',                  # Specify the SLURM queue
    processes=1,                     # Number of processes per job
    cores=1,                          # Number of cores per job
    memory='50GB',                    # Memory per job
    walltime='15:00:00',              # Job duration (hh:mm:ss)
    job_script_prologue=[
        'module load BGC-easybuilded',
        'module load  GCC',
    ]
)

# Scale up the number of workers
#cluster.scale(jobs=20)  # Adjust the number of jobs/workers
cluster.adapt(minimum=0, maximum=30)

# Create a Dask client that connects to the cluster
client = daskClient(cluster)

# Check cluster status
cluster

/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38063 instead
  warnings.warn(


Dashboard: http://10.0.30.56:38063/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.30.56:36175,Workers: 0
Dashboard: http://10.0.30.56:38063/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
import sys
import argparse

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]


args = parser_arguments().parse_args()
args

Namespace(id=None, name=None, index='pei_180', compute_variance=False, region='globe', time_resolution=5, n_components=3, n_samples=100, n_eco_clusters=25, kernel_pca=False, saving_path=None, path_load_experiment=None, method='regional')

In [5]:
import numpy as np 

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]
args = parser_arguments().parse_args()
args.name = "deep_extreme_global"
args.index = "EVI_EN"
args.k_pca = False
args.n_samples = 40000
args.n_components = 3
args.n_eco_clusters = 50
args.compute_variance = False
args.method = "regional"
args.start_year = 2000
args.is_generic_xarray_dataset = False

# args.path_load_experiment = "/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-12_11:52:00_deep_extreme_HR"

LOWER_QUANTILES_LEVEL = np.array([0.01, 0.025, 0.05])
UPPER_QUANTILES_LEVEL = np.array([0.95, 0.975, 0.99])
if args.method == "regional":
    # Apply the regional extremes method
    regional_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
    # regional_extremes_minicube(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "local":
    # Apply the uniform threshold method
    local_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "global":
    raise NotImplementedError("the global method is not yet implemented.")



[2024-12-20 14:38:27] Initialisation of a new model, no path provided for an existing model.
[2024-12-20 14:38:27] The saving path is: /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-20_14:38:27_deep_extreme_global/EVI_EN
[2024-12-20 14:38:27] args saved, path: /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-20_14:38:27_deep_extreme_global/EVI_EN/args.json
[2024-12-20 14:38:27] start of the preprocess
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-20_14:38:27_deep_extreme_global/EVI_EN/temp_file.zarr
[2024-12-20 14:38:27] Data not found at /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-20_14:38:27_deep_extreme_global/EVI_EN/temp_file.zarr
count: 40000
[2024-12-20 14:38:27] Loading dataset...


/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/dask/base.py:1103: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


[########################################] | 100% Completed | 19m 53s


/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 2.04 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


[2024-12-20 15:48:00] temp_file computed and saved.
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-20_14:38:27_deep_extreme_global/EVI_EN/temp_file.zarr
[2024-12-20 15:48:00] Data not found at /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/experiments/2024-12-20_14:38:27_deep_extreme_global/EVI_EN/temp_file.zarr
[2024-12-20 15:48:01] Dataset loaded.


AssertionError: Data not loaded.

In [1]:
client.close()
cluster.close()

NameError: name 'client' is not defined